In [ ]:
import csv
import math
import random

 
def loadCsv():
  lines = csv.reader(open(r'diabetes.csv'))
  dataset = list(lines)
  print(len(dataset)) #converting all string into float
  for i in range(len(dataset)):
    dataset[i] = [float(x) for x in dataset[i]]
  return dataset

def sep(dataset): #separating database according values 1 and 0
  separated = {}
  for i in range(len(dataset)):
    vector = dataset[i]
    if (vector[-1] not in separated):
      separated[vector[-1]] = []
    separated[vector[-1]].append(vector)
  #print(separated)
  return separated

In [ ]:
#calculating mean
def mean(numbers):
  return sum(numbers)/float(len(numbers))

In [ ]:
#calculating standard dev
def stdev(numbers):
  avg = mean(numbers)
  #print ('mean')
  #print(avg)
  variance = sum([pow(x-avg,2) for x in numbers])/float(len(numbers)-1)
  var =math.sqrt(variance)
  #print('variance')
  #print(var)
  return var

In [ ]:
#separating by class
def summarizeByClass(dataset):
  separated = sep(dataset)
  summaries = {}
  for classValue, instances in separated.items():
    #print("****")
    #print(classValue)
    #print("****")
    #print(instances)
    #print(f"number={len(instances)}")
    s = [(mean(i), stdev(i)) for i in zip(*instances)]
    del s[-1]
    summaries[classValue] = s
  return summaries

In [ ]:
#gaussian probability density formula (P(X|C)) the probability after getting evidence posterior probablity
def calculateProbability(x, mean, stdev):
  e = math.exp(-(math.pow(x-mean,2)/(2*math.pow(stdev,2))))
  return (1/(math.sqrt(2*math.pi)*stdev))*e

In [ ]:
#P(C|X) = P(X|C)*P(C)            p(c) = prior probability the probability before getting the evidence 
def calculateClassProbabilities(summaries, inputVector):
  probabilities = {}
  # print(f"s= {summaries}")
  for classValue, classSummaries in summaries.items():
    print(f"cv= {(classValue)}")
    probabilities[classValue] = 1
    for i in range(len(classSummaries)):
      mean, stdev = classSummaries[i]
      x = inputVector[i]
      probabilities[classValue] *= calculateProbability(x, mean, stdev)
  return probabilities

In [ ]:
#import matplotlib.pyplot as plt
def predict(summaries, inputVector):
  probabilities = calculateClassProbabilities(summaries, inputVector)
  bestLabel, bestProb = None, -1
  # print(f"probabilities= {probabilities}")
  probabilities[1]*=(205.0/702.0) #prior prob for class 1
  probabilities[0]*=(497.0/768.0) #prior prob for class 0
  y=1
  print(f"after={probabilities}")
  
  for classValue, probability in probabilities.items(): 
    if bestLabel is None or probability>bestProb:
      bestProb = probability
      bestLabel = classValue
      #plt.scatter(bestLabel,bestProb, label= "scatterplot", color= "green",marker= ".", s=30)
  return bestLabel #returning best probablity

In [ ]:
def main():
  dataset = loadCsv()#loadcsv function
  #testdata set
  testSet = [406.0, -2.169928976, 3.639653999, -4.508497786, 2.730668149, -2.122692871, -2.341016828, -4.235253083, 1.703537658, -1.305279133, -6.716720023, 6.353612319, -8.601648263, 0.449930038, -7.506169374, -0.438081782, -3.694515998, -6.304753386, -1.267587128, 0.35798703, 0.500779054, 0.645103276, -0.503529449, -0.000522822, 0.071695779, 0.092007433, 0.308497932, 0.552590915, 0.298954479, 1.0, 1.0]
  #testSet = [41,	Private,	264663,	Some-college,	10,	Separated,	Prof-specialty,	Own-child,	White,	Female,	0,	3900,	40,	United-States,	<=50K]
  #prepare model
  summaries = summarizeByClass(dataset)
  #test model
  predictions = predict(summaries, testSet)
  print(predictions)
main()

702
cv= 0.0
cv= 1.0
after={0.0: 8.120000277009039e-132, 1.0: 3.0150164439607747e-31}
1.0


In [ ]:
import math
import random
import csv
#import matplotlib.pyplot as plt

def sep(dataset):
	classes = []
	for i in range(len(dataset)):
		if dataset[i][-1] not in classes:
			classes.append(dataset[i][-1])
	for i in range(len(classes)):
		for j in range(len(dataset)):
			if dataset[j][-1] == classes[i]:
				dataset[j][-1] = i
	return dataset		
			

# Splitting the data
def splitting(dataset, ratio):
	train_num = int(len(dataset) * ratio)
	train = []
	# initially testset will have all the dataset
	test = list(dataset)
	while len(train) < train_num:
		# index generated randomly from range 0
		# to length of testset
		index = random.randrange(len(test))
		# from testset, pop data rows and put it in train
		train.append(test.pop(index))
	return train, test


# Group the data rows under each class yes or
# no in dictionary eg: dict[yes] and dict[no]
def groupUnderClass(dataset):
	dict = {}
	for i in range(len(dataset)):
		if (dataset[i][-1] not in dict):
			dict[dataset[i][-1]] = []
		dict[dataset[i][-1]].append(dataset[i])
	return dict


# Calculating Mean
def mean(numbers):
	return sum(numbers) / float(len(numbers))

# Calculating Standard Deviation
def stdev(numbers):
	avg = mean(numbers)
	variance = sum([pow(x - avg, 2) for x in numbers]) / float(len(numbers) - 1)
	return math.sqrt(variance)

def MeanAndStdDev(dataset):
	info = [(mean(attribute), stdev(attribute)) for attribute in zip(*dataset)]
	del info[-1]
	return info

# find Mean and Standard Deviation under each class
def MeanAndStdDevForClass(dataset):
	info = {}
	dict = groupUnderClass(dataset)
	for classValue, instances in dict.items():
		info[classValue] = MeanAndStdDev(instances)
	return info


# Calculate Gaussian Probability Density Function (P(X|C)) prior probablity 
def calculateProbability(x, mean, stdev):
	e = math.exp(-(math.pow(x - mean, 2) / (2 * math.pow(stdev, 2))))
	return (1 / (math.sqrt(2 * math.pi) * stdev)) * e


# Calculate Class Probabilities #P(C|X) = P(X|C)*P(C)            
def calculateClassProbabilities(info, test):
	probabilities = {}
	for classValue, classSummaries in info.items():
		probabilities[classValue] = 1
		for i in range(len(classSummaries)):
			mean, stdev = classSummaries[i]
			x = test[i]
			probabilities[classValue] *= calculateProbability(x, mean, stdev) #posterior probablity
	return probabilities


# Make prediction - highest probability is the prediction
def predict(info, test):
	probabilities = calculateClassProbabilities(info, test)
	bestLabel, bestProb = None, -1
	for classValue, probability in probabilities.items():
		if bestLabel is None or probability > bestProb:
			bestProb = probability
			bestLabel = classValue
	return bestLabel


# returns predictions for a set of examples
def getPredictions(info, test):
	predictions = []
	for i in range(len(test)):
		result = predict(info, test[i])
		predictions.append(result)
	return predictions

# Accuracy score
def accuracy_rate(test, predictions):
	correct = 0
	for i in range(len(test)):
		if test[i][-1] == predictions[i]:
			correct += 1
	return (correct / float(len(test))) * 100.0


filename = r'diabetes.csv'

# load the file and store it in dataset list
dataset = csv.reader(open(filename, "rt"))
dataset = list(dataset)
dataset = sep(dataset)
for i in range(len(dataset)):
	dataset[i] = [float(x) for x in dataset[i]]

	
#split data into two set train_data=80% and test_data=20%
ratio = 0.8
train_data, test_data = splitting(dataset, ratio)
print('len of dataset: ', len(dataset))
print('len of training set: ', len(train_data))
print("len of test set data: ", len(test_data))


info = MeanAndStdDevForClass(train_data)

test_data =  [[159844,	-0.408110817,	3.132944498,	-3.098029764,	5.803892615,	0.890608919,	-0.501474323,	-0.440054283,	0.591828269,	-3.267693259,	-2.223070328,	0.757063214,	-3.501803779,	0.246742224,	-6.06562188,	0.339583134,	-1.005723154,	0.334315624,	0.421261362,	1.247142631,	0.499567913,	0.098482322,	-0.538374705,	-0.217988987,	-1.042656617,	0.314388821,	0.543243576,	0.233851015,	0.1196034,61	,45.51	,1]]
predictions = getPredictions(info, test_data)
print('train data')
print(train_data)
print('test data')
print(test_data)
print('Predictions')
print(predictions)



len of dataset:  702
len of training set:  561
len of test set data:  141
train data
[[25426.0, 1.125335535, 1.130146484, -0.962975335, 2.675687859, 0.990074518, -0.243317741, 0.316192138, 0.122960132, -1.143342795, -0.369908966, 1.834494198, -0.491243315, -1.129907005, -2.238622244, 0.278468742, 1.477925603, 1.776454295, 1.149282186, -1.306994645, -0.138813698, -0.166736918, -0.521933839, -0.112376257, -0.59207675, 0.520791314, 0.043353581, 0.015159021, 0.063611722, 3.76, 1.0], [313.0, -1.038356228, 0.756121842, 0.719941776, 0.988401807, -0.237289725, -0.378861992, 0.413667984, 0.436786746, -0.448178491, -0.346581608, -0.786498013, -0.999026789, -1.968015268, 1.049901672, 1.489075793, -0.372646824, 0.350349838, -0.035845284, 0.110459559, -0.344187258, 0.189549653, 0.477872319, 0.298491951, 0.029367906, -0.30041994, -0.294100311, -0.009904639, 0.038860559, 68.0, 0.0], [13.0, -0.436905071, 0.918966213, 0.924590774, -0.727219054, 0.915678718, -0.127867352, 0.707641607, 0.087962355, -0.66